### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
slice_name="l4s-" + fablib.get_bastion_username()

node_conf = [
 {'name': "tx0",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "tx1",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "router", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "delay", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "rx0",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "rx1",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}
]
net_conf = [
 {"name": "net-tx", "subnet": "10.0.0.0/24", "nodes": [{"name": "tx0",   "addr": "10.0.0.100"}, {"name": "tx1",   "addr": "10.0.0.101"}, {"name": "delay", "addr": "10.0.0.2"}]},
 {"name": "net-delay-router", "subnet": "10.0.2.0/24", "nodes": [{"name": "delay",   "addr": "10.0.2.2"}, {"name": "router", "addr": "10.0.2.1"}]},
 {"name": "net-rx", "subnet": "10.0.5.0/24", "nodes": [{"name": "router",   "addr": "10.0.5.1"}, {"name": "rx0", "addr": "10.0.5.100"}, {"name": "rx1", "addr": "10.0.5.101"}]}

]
route_conf = [
 {"addr": "10.0.5.0/24", "gw": "10.0.0.2", "nodes": ["tx0", "tx1"]}, 
 {"addr": "10.0.5.0/24", "gw": "10.0.2.1", "nodes": ["delay"]},

 {"addr": "10.0.0.0/24", "gw": "10.0.5.1", "nodes": ["rx0", "rx1"]},
 {"addr": "10.0.0.0/24", "gw": "10.0.2.2", "nodes": ["router"]}

]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Extend your slice

If you don’t plan to finish an experiment in one day, you can extend your slice. The following cell extends your reservation for 7 days.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

### Configure Experiment

In [ ]:
for node in slice.get_nodes():
    # Download and unzip the kernel package
    node.execute("wget https://github.com/L4STeam/linux/releases/download/testing-build/l4s-testing.zip")
    node.execute("sudo apt install unzip")
    node.execute("unzip l4s-testing.zip")
    
    # Install the kernel packages and update GRUB
    node.execute("sudo dpkg --install debian_build/*")
    node.execute("sudo update-grub")
    node.execute("sudo reboot")

# wait for all nodes to come back up
slice.wait_ssh(progress=True)
for node in slice.get_nodes():
    # check kernel version
    node.execute("hostname; uname -a")

In [ ]:
#inital configuration for the senders
slice.get_node(name="tx0").execute("sudo modprobe tcp_prague")
slice.get_node(name="tx0").execute("sudo sysctl -w net.ipv4.tcp_congestion_control=prague")
slice.get_node(name="tx0").execute("sudo sysctl -w net.ipv4.tcp_ecn=3")

slice.get_node(name="tx1").execute("sudo sysctl -w net.ipv4.tcp_congestion_control=cubic")
slice.get_node(name="tx1").execute("sudo sysctl -w net.ipv4.tcp_ecn=1")

In [ ]:
#configuration for DUALPI2 bottleneck
cmd_dualpi2="""sudo apt-get update
sudo apt -y install git gcc make bison flex libdb-dev libelf-dev pkg-config libbpf-dev libmnl-dev libcap-dev libatm1-dev selinux-utils libselinux1-dev
sudo git clone https://github.com/L4STeam/iproute2.git && cd iproute2
sudo ./configure
sudo make
sudo make install"""
slice.get_node(name="router").execute(cmd_dualpi2)
slice.get_node(name="router").execute("sudo modprobe sch_dualpi2")

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Execute Experiment

In [ ]:
# nodes and instances

tx0_node = slice.get_node(name="tx0")
tx1_node = slice.get_node(name="tx1")
rx0_node = slice.get_node(name="rx0")
rx1_node = slice.get_node(name="rx1")
delay_node = slice.get_node(name="delay")
router_node = slice.get_node(name="router")
# interfaces

tx0_egress_iface  = tx0_node.get_interface(network_name = "net-tx")
tx1_egress_iface  = tx1_node.get_interface(network_name = "net-tx")

delay_ingress_tx_iface  = delay_node.get_interface(network_name = "net-tx")
delay_egress_iface  = delay_node.get_interface(network_name = "net-delay-router")
delay_ingress_tx_name = delay_ingress_tx_iface.get_device_name()
delay_egress_name = delay_egress_iface.get_device_name()

router_ingress_iface  = router_node.get_interface(network_name = "net-delay-router")
router_egress_iface  = router_node.get_interface(network_name = "net-rx")
router_egress_name  = router_egress_iface.get_device_name()


rx0_ingress_iface  = rx0_node.get_interface(network_name = "net-rx")
rx1_ingress_iface  = rx1_node.get_interface(network_name = "net-rx")

In [ ]:
# generate full factorial experiment
import itertools

exp_factors = {
    'n_bdp': [0.5, 2, 5, 10],  # n x bandwidth delay product
    'btl_capacity': [100, 1000],
    'base_rtt': [5, 10, 50, 100],
    'aqm': ['FIFO', 'single_queue_FQ', 'Codel', 'FQ', 'FQ_Codel', 'DualPI2'],
    #'aqm': ['single_queue_FQ', 'Codel', 'FQ', 'FQ_Codel', 'DualPI2'],
    'ecn_threshold': [1, 5, 20],
    'ecn_fallback': [0, 1],  #fallback algorithm, it falls back when it detects single queue classic ECN bottleneck # 0: OFF, 1: ON
    'rx0_ecn': [0, 1, 2],  # 0: noecn, 1: ecn, 2: accecn
    'rx1_ecn': [0, 1],  # 0: noecn, 1: ecn
    'cc_tx0': ["prague"],
    'cc_tx1': ["cubic"],
    'trial': [1, 2, 3, 4, 5]
}

factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))

exp_lists = []

seen_combinations = set()

for factor_l in factor_lists:
    temp_dict = dict(zip(factor_names, factor_l))
    if temp_dict['n_bdp'] * temp_dict['base_rtt'] >= temp_dict['ecn_threshold']:
        if temp_dict['aqm'] == 'FIFO':
            del temp_dict['ecn_threshold']
        # Convert dict to a frozenset for set operations
        fs = frozenset(temp_dict.items())
    
        if fs not in seen_combinations:
            seen_combinations.add(fs)
            exp_lists.append(temp_dict)

data_dir_tx0 = slice_name + 'singlebottleneck'+"-tx0"
data_dir_tx1 = slice_name + 'singlebottleneck'+"-tx1"

print("Number of experiments:",len(exp_lists))

In [ ]:
# run experiments
import time
d = 60 #duration in seconds

em = [delay_ingress_tx_name, delay_egress_name]

commands_noecn='''
sudo sysctl -w net.ipv4.tcp_congestion_control=cubic  
sudo sysctl -w net.ipv4.tcp_ecn=0''' #tcp_prague, no ECN

commands_ecn='''
sudo sysctl -w net.ipv4.tcp_congestion_control=cubic  
sudo sysctl -w net.ipv4.tcp_ecn=1''' #tcp_prague, ECN

commands_accecn='''
sudo sysctl -w net.ipv4.tcp_congestion_control=prague  
sudo sysctl -w net.ipv4.tcp_ecn=3''' #tcp_prague, AccECN

server_ecn_list=[commands_noecn, commands_ecn, commands_accecn]

for exp in exp_lists:

    # check if we already ran this experiment
    # (allow stop/resume)
    name_tx0="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % (exp['cc_tx0'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'], exp['trial'])
    name_tx1="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % (exp['cc_tx1'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'], exp['trial'])
    
    file_out_tx0_json = name_tx0+"-result.json"
    file_out_tx0_ss = name_tx0+"-ss.txt"
    stdout_tx0_json, stderr_tx0_json = tx0_node.execute("ls " + file_out_tx0_json, quiet=True) 
    stdout_tx0_ss, stderr_tx0_ss = tx0_node.execute("ls " + file_out_tx0_ss, quiet=True)
    
    file_out_tx1_json =name_tx1+"-result.json"
    file_out_tx1_ss = name_tx1+"-ss.txt"
    stdout_tx1_json, stderr_tx1_json = tx1_node.execute("ls " + file_out_tx1_json, quiet=True) 
    stdout_tx1_ss, stderr_tx1_ss = tx1_node.execute("ls " + file_out_tx1_ss, quiet=True) 

    if len(stdout_tx0_json) and len(stdout_tx0_ss) and len(stdout_tx1_json) and len(stdout_tx1_ss):
        print("Already have " + name_tx0 + " and "+ name_tx1 + ", skipping")

    elif len(stderr_tx0_json) or len(stderr_tx0_ss) or len(stderr_tx1_json) or len(stderr_tx1_ss):
        print("Running experiment to generate " + name_tx0 + " and "+ name_tx1) 
        
        # delay at emulator
        for e in em:
            cmds = "sudo tc qdisc replace dev {iface} root netem delay {owd}ms limit 60000".format(iface=e, owd=exp['base_rtt']/2)
            delay_node.execute(cmds)
        
        # fixed values
        btl_limit    = int(1000*exp['n_bdp']*exp['btl_capacity']*exp['base_rtt']/8) # limit of the bottleneck, n_bdp x BDP in bytes 
        packet_number=int(btl_limit/1500)+1
        
        
        #ecn-fallback configuration
               
        commands = "cd /sys/module/tcp_prague/parameters && echo {value} | sudo tee prague_ecn_fallback".format(value=str(exp['ecn_fallback']))
        tx0_node.execute(commands)
        
        #receiver ecn configuration
        rx0_node.execute(server_ecn_list[exp['rx0_ecn']])
        rx1_node.execute(server_ecn_list[exp['rx1_ecn']])
        
        #aqm type selection
        cmds_prefix = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3 
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit 
            '''.format(iface=router_egress_name, capacity=exp['btl_capacity'], buffer=btl_limit)
        cmds_specific = {
        'FIFO': "sudo tc qdisc add dev {iface} parent 1:3 handle 3: bfifo limit {buffer}".format(iface=router_egress_name, buffer=btl_limit),
        'single_queue_FQ': "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: fq limit {packet_limit} flow_limit {packet_limit} orphan_mask 0 ce_threshold {threshold}ms".format(iface=router_egress_name, packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'Codel': "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: codel limit {packet_limit} target {target}ms interval 100ms ecn ce_threshold {threshold}ms".format(iface=router_egress_name, packet_limit=packet_number, target=exp['base_rtt']*exp['n_bdp'], threshold=exp.get('ecn_threshold', 0)),
        'FQ': "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: fq limit {packet_limit} flow_limit {packet_limit} ce_threshold {threshold}ms".format(iface=router_egress_name, packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'FQ_Codel': "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: fq_codel limit {packet_limit} target {target}ms interval 100ms ecn ce_threshold {threshold}ms".format(iface=router_egress_name, packet_limit=packet_number, target=exp['base_rtt']*exp['n_bdp'], threshold=exp.get('ecn_threshold', 0)),
        'DualPI2': "sudo tc qdisc add dev {iface} parent 1:3 handle 3: dualpi2 target {threshold}ms".format(iface=router_egress_name, threshold=exp.get('ecn_threshold', 0))
        }

        router_node.execute(cmds_prefix)
        router_node.execute(cmds_specific[ exp['aqm'] ])
            
        rx0_node.execute("killall iperf3")
        rx1_node.execute("killall iperf3")
        
        ss_tx0_script="rm -f {flow}-ss.txt; start_time=$(date +%s); while true; do ss --no-header -eipn dst 10.0.5.100 | ts '%.s' | tee -a {flow}-ss.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.1; done;"
        ss_tx1_script="rm -f {flow}-ss.txt; start_time=$(date +%s); while true; do ss --no-header -eipn dst 10.0.5.101 | ts '%.s' | tee -a {flow}-ss.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.1; done;"

        #print("Starting experiment with {1} bdp {2} capacity {3} rtt {4} {5} thrshold {6} ecn_fallback {7} rx0 {8} rx1 for {duration} seconds".format(duration=d, 1=exp['n_bdp'], 2=exp['btl_capacity'], 3=exp['base_rtt'], 4=exp['aqm'], 5=exp['ecn_threshold'], 6= exp['ecn_fallback'], 7=exp['rx0_ecn'], 8=exp['rx1_ecn']))
        
        rx0_node.execute("iperf3 -s -1 -p 4000 -D")
        rx1_node.execute("iperf3 -s -1 -p 5000 -D")
        
        tx0_node.execute_thread(ss_tx0_script.format(flow=name_tx0, duration=d))
        tx1_node.execute_thread(ss_tx1_script.format(flow=name_tx1, duration=d))
        
        tx0_node.execute_thread("sleep 1; iperf3 -c 10.0.5.100 -t {duration} -P {flows} -C {cc} -p 4000 -J > {flow}-result.json".format(flow =name_tx0, duration=d, flows=1, cc=exp['cc_tx0']))
        stdout, stderr = tx1_node.execute("sleep 1; iperf3 -c 10.0.5.101 -t {duration} -P {flows} -C {cc} -p 5000 -J > {flow}-result.json".format(flow =name_tx1, duration=d, flows=1, cc=exp['cc_tx1']))
        time.sleep(3)  # time.sleep(1)
        
print("finished")
        
        

### Analysis of the results

In [ ]:
for exp in exp_lists:
    name_tx0="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d" % (exp['cc_tx0'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'])
    name_tx1="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d" % (exp['cc_tx1'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'])
    
    
    file_out_tx0_csv = name_tx0+"-ss.csv"
    stdout_tx0_csv, stderr_tx0_csv = tx0_node.execute("ls " + file_out_tx0_csv, quiet=True) 
    
    file_out_tx1_csv = name_tx1+"-ss.csv"
    stdout_tx1_csv, stderr_tx1_csv = tx1_node.execute("ls " + file_out_tx1_csv, quiet=True) 

    if len(stdout_tx0_csv) and len(stdout_tx1_csv):
        print("Already have " + name_tx0 + " and "+ name_tx1 + ", skipping")

    elif len(stderr_tx0_csv) or len(stderr_tx1_csv):
        print("Running to generate csv files " + name_tx0 + " and "+ name_tx1)
        
        
    #tx0_node.download_file("/home/fabric/work/{flow}-result.json".format(flow=name_prague),"/home/ubuntu/{f_type}-result.json".format(f_type=name_prague))
    #tx1_node.download_file("/home/fabric/work/{flow}-result.json".format(flow=name_cubic),"/home/ubuntu/{f_type}-result.json".format(f_type=name_cubic))
    #tx0_node.download_file("/home/fabric/work/{flow}-ss.txt".format(flow=name_prague),"/home/ubuntu/{f_type}-ss.txt".format(f_type=name_prague))
    #tx1_node.download_file("/home/fabric/work/{flow}-ss.txt".format(flow=name_cubic),"/home/ubuntu/{f_type}-ss.txt".format(f_type=name_cubic))


    
        ss_tx0_script_processing="""

        f_1={types}; 
        rm -f ${{f_1}}-ss.csv;
        cat ${{f_1}}-ss.txt | sed -e ":a; /<->$/ {{ N; s/<->\\n//; ba; }}"  | grep "iperf3" | grep -v "SYN-SENT"> ${{f_1}}-ss-processed.txt; 
        cat ${{f_1}}-ss-processed.txt | awk '{{print $1}}' > ts-${{f_1}}.txt; 
        cat ${{f_1}}-ss-processed.txt | grep -oP '\\bcwnd:.*?(\s|$)' | awk -F '[:,]' '{{print $2}}' | tr -d ' ' > cwnd-${{f_1}}.txt; 
        cat ${{f_1}}-ss-processed.txt | grep -oP '\\brtt:.*?(\s|$)' | awk -F '[:,]' '{{print $2}}' | tr -d ' '  | cut -d '/' -f 1   > srtt-${{f_1}}.txt; 
        cat ${{f_1}}-ss-processed.txt | grep -oP '\\bfd=.*?(\s|$)' | awk -F '[=,]' '{{print $2}}' | tr -d ')' | tr -d ' '   > fd-${{f_1}}.txt;
        paste ts-${{f_1}}.txt fd-${{f_1}}.txt cwnd-${{f_1}}.txt srtt-${{f_1}}.txt -d ',' > ${{f_1}}-ss.csv;""".format(types=name_tx0)
     
    
        tx0_node.execute(ss_tx0_script_processing)


        ss_tx1_script_processing="""

        f_2={types};
        rm -f ${{f_2}}-ss.csv;
        cat ${{f_2}}-ss.txt | sed -e ":a; /<->$/ {{ N; s/<->\\n//; ba; }}"  | grep "iperf3" | grep -v "SYN-SENT" > ${{f_2}}-ss-processed.txt; 
        cat ${{f_2}}-ss-processed.txt | awk '{{print $1}}' > ts-${{f_2}}.txt; 
        cat ${{f_2}}-ss-processed.txt | grep -oP '\\bcwnd:.*?(\s|$)' |  awk -F '[:,]' '{{print $2}}' | tr -d ' ' > cwnd-${{f_2}}.txt; 
        cat ${{f_2}}-ss-processed.txt | grep -oP '\\brtt:.*?(\s|$)' |  awk -F '[:,]' '{{print $2}}' | tr -d ' '  | cut -d '/' -f 1   > srtt-${{f_2}}.txt; 
        cat ${{f_2}}-ss-processed.txt | grep -oP '\\bfd=.*?(\s|$)' |  awk -F '[=,]' '{{print $2}}' | tr -d ')' | tr -d ' '   > fd-${{f_2}}.txt;
        paste ts-${{f_2}}.txt fd-${{f_2}}.txt cwnd-${{f_2}}.txt srtt-${{f_2}}.txt -d ',' > ${{f_2}}-ss.csv;""".format(types=name_tx1)


        tx1_node.execute(ss_tx1_script_processing)

    #tx0_node.download_file("/home/fabric/work/{f_type}-ss.csv".format(f_type=name_prague),"/home/ubuntu/{f_type}-ss.csv".format(f_type=name_prague))
    #tx1_node.download_file("/home/fabric/work/{f_type}-ss.csv".format(f_type=name_cubic),"/home/ubuntu/{f_type}-ss.csv".format(f_type=name_cubic))



#tx0_node.execute('rm -r '+data_dir_tx0)
tx0_node.execute('mkdir '+data_dir_tx0)

tx0_node.execute('mv *.json '+ data_dir_tx0)
tx0_node.execute('mv *.txt '+ data_dir_tx0)
tx0_node.execute('mv *.csv '+ data_dir_tx0)

tx0_node.execute('tar -czvf '+data_dir_tx0+ '.tgz ' +  data_dir_tx0)
#tx0_node.download_file(data_dir_tx0+'.tgz ', '/home/ubuntu/' + data_dir_tx0+ '.tgz')


#tx1_node.execute('rm -r '+data_dir_tx1)
tx1_node.execute('mkdir '+data_dir_tx1)

tx1_node.execute('mv *.json '+ data_dir_tx1)
tx1_node.execute('mv *.txt '+ data_dir_tx1)
tx1_node.execute('mv *.csv '+ data_dir_tx1)
        
tx1_node.execute('tar -czvf '+data_dir_tx1+ '.tgz ' +  data_dir_tx1)
#tx1_node.download_file(data_dir_tx1+'.tgz ', '/home/ubuntu/' + data_dir_tx1+ '.tgz')

In [ ]:
import shutil, tarfile

# remove previously stored data files
#shutil.rmtree(data_dir_tx0)
#shutil.rmtree(data_dir_tx1)

# extract tar files 
with tarfile.open(data_dir_tx0+'.tgz ', 'r:gz') as tar:
    tar.extractall()
    
with tarfile.open(data_dir_tx1+'.tgz ', 'r:gz') as tar:
    tar.extractall()

### You can download the .tgz file from the node to the local machine and use plot_on_local_barplot.py file to see the results.

In [ ]:
import json

import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

import re

from statistics import mean


#get all throughput and SRTT data

throughput_data = {}  # Initialize the dictionary

srtt_data = {}

for exp in exp_lists:

    name_tx0="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d" % (exp['cc_tx0'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'])
    name_tx1="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d" % (exp['cc_tx1'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'])
    

    # Load the JSON output file into a Python object
    
    ##### Average Throughput for Each Flow ****
    
    with open("/home/fabric/work/"+data_dir_tx0+"/{flow1}-result.json".format(flow1=name_tx0), "r") as f:
        iperf3_data = json.load(f)

    throughput_data[name_tx0]  = iperf3_data['end']['sum_received']['bits_per_second']/(1000000*1) # to convert Mbit


    with open("/home/fabric/work/"+data_dir_tx1+"/{flow1}-result.json".format(flow1=name_tx1), "r") as f:
        iperf3_data = json.load(f)


    throughput_data[name_tx1]  = iperf3_data['end']['sum_received']['bits_per_second']/(1000000*1) # to convert Mbit
    
    
    
    ##### Average SRTT for Each Flow ******
    
    columns = ['timestamp', 'flow ID', 'cwnd', 'srtt']
    df_f1= pd.read_csv('/home/fabric/work/'+data_dir_tx0+'/{flow1}-ss.csv'.format(flow1=name_tx0), names=columns)
    df_f2= pd.read_csv('/home/fabric/work/'+data_dir_tx1+'/{flow1}-ss.csv'.format(flow1=name_tx1), names=columns)
    
    # Filter out rows with flow ID = 4, they are for the control flows
    df_f1= df_f1[df_f1['flow ID'] != 4]
    df_f2= df_f2[df_f2['flow ID'] != 4]
    
    average_RTT_f1 = df_f1['srtt'].mean()
    average_RTT_f2 = df_f2['srtt'].mean()
    
    srtt_data[name_tx0] = average_RTT_f1
    srtt_data[name_tx1] = average_RTT_f2

import matplotlib.pyplot as plt
import numpy as np

# Placeholder: A dictionary to store throughput data for each experiment
# Example format: "n_bdp_btl_capacity_base_rtt_aqm_ecn_threshold_ecn_fallback_rx_ecn_cc_tx_trial"
# throughput_data = {
#     "prague_0.5_100_10_FIFO_none_0_0_0": 55.787676690306085,
#     "prague_2.0_100_10_FIFO_none_0_0_0": 54.248112443427175,
#     "prague_5.0_100_10_FIFO_none_0_0_0": 38.2013028326071,
#     "prague_10.0_100_10_FIFO_none_0_0_0": 63.37023092922114,
#     "cubic_0.5_100_10_FIFO_none_0_0_0": 47.47733567404541,
#     "cubic_2.0_100_10_FIFO_none_0_0_0": 37.5916393558464,
#     "cubic_5.0_100_10_FIFO_none_0_0_0": 65.17600721037583,
#     "cubic_10.0_100_10_FIFO_none_0_0_0": 50.3362326648659,
# }


# User-specified parameters
specified_params = {
    'btl_capacity': 100,
    'n_bdp': 2,
    'base_rtt': 10,
    'ecn_threshold': 5,
    'ecn_fallback': 0,
    'rx0_ecn': 0,
    'rx1_ecn': 0
    
}

factor_x = 'aqm' 

relevant_data_tx0 = {}
relevant_data_tx1 = {}

relevant_srtt_data_tx0 = {}
relevant_srtt_data_tx1 = {}

for exp in exp_lists:
    #is_relevant = all(item in exp.items() for item in specified_params.items())
    
    is_relevant = all(
        (k == 'ecn_threshold' and (v == exp.get(k) or exp.get(k) is None)) or 
        (k != 'ecn_threshold' and v == exp.get(k)) for k, v in specified_params.items()
    )

    if is_relevant:
        name_tx0="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d" % (exp['cc_tx0'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'])
        name_tx1="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d" % (exp['cc_tx1'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'])

        print(exp[factor_x])
        if name_tx0 in throughput_data:
            xval = exp[factor_x]
            if xval not in relevant_data_tx0:
                relevant_data_tx0[xval] = []
            relevant_data_tx0[xval].append(throughput_data[name_tx0])
            
        if name_tx1 in throughput_data:
            xval = exp[factor_x]
            if xval not in relevant_data_tx1:
                relevant_data_tx1[xval] = []
            relevant_data_tx1[xval].append(throughput_data[name_tx1])
            
        if name_tx0 in srtt_data:
            xval = exp[factor_x]
            if xval not in relevant_srtt_data_tx0:
                relevant_srtt_data_tx0[xval] = []
            relevant_srtt_data_tx0[xval].append(srtt_data[name_tx0])
            
        if name_tx1 in srtt_data:
            xval = exp[factor_x]
            if xval not in relevant_srtt_data_tx1:
                relevant_srtt_data_tx1[xval] = []
            relevant_srtt_data_tx1[xval].append(srtt_data[name_tx1])
            
        

# Average the throughputs over all trials for factor_x
for xval, throughputs in relevant_data_tx0.items():
    relevant_data_tx0[xval] = np.mean(throughputs)
    
for xval, throughputs in relevant_data_tx1.items():
    relevant_data_tx1[xval] = np.mean(throughputs)
    
for xval, srtts in relevant_srtt_data_tx0.items():
    relevant_srtt_data_tx0[xval] = np.mean(srtts)
    
for xval, srtts in relevant_srtt_data_tx1.items():
    relevant_srtt_data_tx1[xval] = np.mean(srtts)


# Sort values
xvals = sorted(list(set(list(relevant_data_tx0.keys()) + list(relevant_data_tx1.keys()))))

print(xvals)

# Get throughputs for sorted values
throughputs_tx0 = [relevant_data_tx0.get(xval, 0) for xval in xvals]
throughputs_tx1 = [relevant_data_tx1.get(xval, 0) for xval in xvals]

rtts_tx0 = [relevant_srtt_data_tx0.get(xval, 0) for xval in xvals]
rtts_tx1 = [relevant_srtt_data_tx1.get(xval, 0) for xval in xvals]

print(rtts_tx0)
print(rtts_tx1)
bar_width = 0.35
index = np.arange(len(xvals))

plt.figure(figsize=(10,6))
bar1 = plt.bar(index, throughputs_tx0, bar_width, label='TX0', alpha=0.8, color='b')
bar2 = plt.bar(index + bar_width, throughputs_tx1, bar_width, label='TX1', alpha=0.8, color='r')

# Annotate bars for TX0
for bar in bar1:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.5, f"{height:.2f}", ha='center', va='bottom')

# Annotate bars for TX1
for bar in bar2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.5, f"{height:.2f}", ha='center', va='bottom')

# Label the bars and the x-axis
plt.xlabel(factor_x)
plt.ylabel('Average Throughput')
plt.title('Average Throughput vs queue type for different flows')
plt.xticks(index + bar_width/2, xvals)  # Positioning on the x axis
plt.legend()
plt.tight_layout()

plt.show()



bar_width = 0.35
index = np.arange(len(xvals))

plt.figure(figsize=(10,6))
bar1 = plt.bar(index, rtts_tx0, bar_width, label='TX0', alpha=0.8, color='b')
bar2 = plt.bar(index + bar_width, rtts_tx1, bar_width, label='TX1', alpha=0.8, color='r')

# Annotate bars for TX0
for bar in bar1:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.5, f"{height:.2f}", ha='center', va='bottom')

# Annotate bars for TX1
for bar in bar2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.5, f"{height:.2f}", ha='center', va='bottom')

# Label the bars and the x-axis
plt.xlabel(factor_x)
plt.ylabel('Average SRTT')
plt.title('Average SRTT vs queue type for different flows')
plt.xticks(index + bar_width/2, xvals)  # Positioning on the x axis
plt.legend()
plt.tight_layout()

plt.show()
